In [0]:
# /FileStore/tables/cars-1.csv

# /FileStore/tables/usuarios.csv

**Creando sesion en spark**

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lit, year, sum, when, desc, max, min, expr, row_number

SpSession = SparkSession.builder.appName('Demo Spark').getOrCreate()
print(spark)

In [0]:
SpContext = SpSession.sparkContext

**Carga de datos**

In [0]:
df = SpSession.read.csv('/FileStore/tables/cars-1.csv', header=True, sep=';', inferSchema=True)
df.show(12)

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0|  4341|        10.0|   70|    US|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0|  4354|         9.0|   70|    US|
|   Plymouth Fury iii|14.0|        8|       440.0|

**Conociendo la data**

In [0]:
df.printSchema()

root
 |-- Car: string (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: double (nullable = true)
 |-- Weight: decimal(4,0) (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Model: integer (nullable = true)
 |-- Origin: string (nullable = true)



**Seleccion de columnas**

In [0]:
# Primer metodo

df.select(df.Origin).show(truncate = False)

# Segundo metodo

df.select(df['Origin']).show(truncate = False)

# Tercer metodo

from pyspark.sql.functions import col
df.select(col('Origin')).show(truncate = False)

#                                       seleccion de multiples columnas

# Primer metodo

#df.select(df.Origin, df.Cylinders).show(truncate = False)

# Segundo metodo

#df.select(df['Origin'], df['Cylinders']).show(truncate = False)

# Tercer metodo

#df.select(col('Origin'), col('Cylinders')).show(truncate = False)

+------+
|Origin|
+------+
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|Europe|
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
+------+
only showing top 20 rows

+------+
|Origin|
+------+
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|Europe|
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
+------+
only showing top 20 rows

+------+
|Origin|
+------+
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|Europe|
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
|US    |
+------+
only showing top 20 rows



**Agregar nuevas columnas**

In [0]:
from pyspark.sql.functions import lit

df = df.withColumn('Color',lit('Red'))
df = df.withColumn('Modelo',lit('2019')).withColumn('Estado',lit('Unico dueño'))
df.show(5, truncate = True)

+--------------------+----+---------+------------+----------+------+------------+-----+------+-----+------+-----------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Color|Modelo|     Estado|
+--------------------+----+---------+------------+----------+------+------------+-----+------+-----+------+-----------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|  Red|  2019|Unico dueño|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|  Red|  2019|Unico dueño|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|  Red|  2019|Unico dueño|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|  Red|  2019|Unico dueño|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|  Red|  2019|Unico dueño|
+--------------------+----+---------+---

**Agrupar columnas**

In [0]:
df.groupBy('Origin').count().show(truncate = False)
df.groupBy('Origin','Horsepower').count().show(truncate = False)

+------+-----+
|Origin|count|
+------+-----+
|Europe|73   |
|US    |254  |
|Japan |79   |
+------+-----+

+------+----------+-----+
|Origin|Horsepower|count|
+------+----------+-----+
|Japan |52.0      |2    |
|US    |225.0     |3    |
|US    |190.0     |3    |
|US    |78.0      |2    |
|Japan |58.0      |1    |
|US    |90.0      |15   |
|US    |149.0     |1    |
|US    |66.0      |1    |
|Europe|81.0      |1    |
|US    |150.0     |22   |
|US    |220.0     |1    |
|US    |112.0     |1    |
|Europe|62.0      |1    |
|Europe|86.0      |1    |
|Europe|78.0      |4    |
|Europe|88.0      |3    |
|US    |81.0      |1    |
|Japan |62.0      |1    |
|US    |72.0      |5    |
|US    |80.0      |6    |
+------+----------+-----+
only showing top 20 rows



**Eliminar columnas**

In [0]:
df.drop('Color').show(5)

df

+--------------------+----+---------+------------+----------+------+------------+-----+------+------+-----------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Modelo|     Estado|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------+-----------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0|  3504|        12.0|   70|    US|  2019|Unico dueño|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0|  3693|        11.5|   70|    US|  2019|Unico dueño|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0|  3436|        11.0|   70|    US|  2019|Unico dueño|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0|  3433|        12.0|   70|    US|  2019|Unico dueño|
|         Ford Torino|17.0|        8|       302.0|     140.0|  3449|        10.5|   70|    US|  2019|Unico dueño|
+--------------------+----+---------+------------+----------+------+------------+-----+-

**Ordenar por columnas**

In [0]:
df.orderBy('Horsepower').show(truncate = False)

df.orderBy('Horsepower', ascending = False).show(truncate = False)

+-------------------------------+----+---------+------------+----------+------+------------+-----+------+-----+------+-----------+
|Car                            |MPG |Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Color|Modelo|Estado     |
+-------------------------------+----+---------+------------+----------+------+------------+-----+------+-----+------+-----------+
|Ford Pinto                     |25.0|4        |98.0        |0.0       |2046  |19.0        |71   |US    |Red  |2019  |Unico dueño|
|Ford Maverick                  |21.0|6        |200.0       |0.0       |2875  |17.0        |74   |US    |Red  |2019  |Unico dueño|
|Renault Lecar Deluxe           |40.9|4        |85.0        |0.0       |1835  |17.3        |80   |Europe|Red  |2019  |Unico dueño|
|Ford Mustang Cobra             |23.6|4        |140.0       |0.0       |2905  |14.3        |80   |US    |Red  |2019  |Unico dueño|
|Renault 18i                    |34.5|4        |100.0       |0.0       |2320  |15.8

**Filtrado de datos**

In [0]:
df.filter(col('Horsepower')=='0').show()

df.filter((col('Horsepower')=='0')&(col('Model')=='80')).show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+-----+------+-----------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|Color|Modelo|     Estado|
+--------------------+----+---------+------------+----------+------+------------+-----+------+-----+------+-----------+
|          Ford Pinto|25.0|        4|        98.0|       0.0|  2046|        19.0|   71|    US|  Red|  2019|Unico dueño|
|       Ford Maverick|21.0|        6|       200.0|       0.0|  2875|        17.0|   74|    US|  Red|  2019|Unico dueño|
|Renault Lecar Deluxe|40.9|        4|        85.0|       0.0|  1835|        17.3|   80|Europe|  Red|  2019|Unico dueño|
|  Ford Mustang Cobra|23.6|        4|       140.0|       0.0|  2905|        14.3|   80|    US|  Red|  2019|Unico dueño|
|         Renault 18i|34.5|        4|       100.0|       0.0|  2320|        15.8|   81|Europe|  Red|  2019|Unico dueño|
|      AMC Concord DL|23.0|        4|   

# **Ejercicios de practica**

**1. Filtrar los autos que tienen más de 6 cilindros.**

In [0]:


df.filter(col('Cylinders')>6).display()


Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado
Chevrolet Chevelle Malibu,18.0,8,307.0,130.0,3504,12.0,70,US,Red,2019,Unico dueño
Buick Skylark 320,15.0,8,350.0,165.0,3693,11.5,70,US,Red,2019,Unico dueño
Plymouth Satellite,18.0,8,318.0,150.0,3436,11.0,70,US,Red,2019,Unico dueño
AMC Rebel SST,16.0,8,304.0,150.0,3433,12.0,70,US,Red,2019,Unico dueño
Ford Torino,17.0,8,302.0,140.0,3449,10.5,70,US,Red,2019,Unico dueño
Ford Galaxie 500,15.0,8,429.0,198.0,4341,10.0,70,US,Red,2019,Unico dueño
Chevrolet Impala,14.0,8,454.0,220.0,4354,9.0,70,US,Red,2019,Unico dueño
Plymouth Fury iii,14.0,8,440.0,215.0,4312,8.5,70,US,Red,2019,Unico dueño
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño
AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70,US,Red,2019,Unico dueño


**2. Calcular el promedio de millas por galón (MPG) por cada número de cilindros.**

In [0]:
from pyspark.sql.functions import avg

num_cylinders = df.groupBy('Cylinders').agg(avg('MPG'))

display(num_cylinders)

Cylinders,avg(MPG)
6,19.985714285714284
3,20.55
5,27.366666666666664
4,28.862318840579714
8,14.270370370370372


**3. Contar cuántos autos hay por origen.**

In [0]:

#Agrupar por origen

car_origin = df.groupBy('Origin').count()

display(car_origin)

Origin,count
Europe,73
US,254
Japan,79


**4. Encontrar el auto con mayor caballaje (Horsepower) y mostrar su nombre y origen.**

In [0]:
#Mostrar el numero mas alto de caballos de fuerza
max_horsePower = df.select(max(col('Horsepower'))).first()[0]
#Filtrar la columna para que muestre el carro con mayor caballo de fuerza
car = df.filter(col('Horsepower')==max_horsePower)
#Mostrar solo la columna nombre, origen y caballos
resultado = car.select('car','origin','Horsepower')
display(resultado)

car,origin,Horsepower
Pontiac Grand Prix,US,230.0


**5. Seleccionar solo las columnas "Car", "MPG" y "Horsepower", y mostrar los primeros 10 registros.**

In [0]:
df.select('Car','MPG','Horsepower').limit(10).display()

Car,MPG,Horsepower
Chevrolet Chevelle Malibu,18.0,130.0
Buick Skylark 320,15.0,165.0
Plymouth Satellite,18.0,150.0
AMC Rebel SST,16.0,150.0
Ford Torino,17.0,140.0
Ford Galaxie 500,15.0,198.0
Chevrolet Impala,14.0,220.0
Plymouth Fury iii,14.0,215.0
Pontiac Catalina,14.0,225.0
AMC Ambassador DPL,15.0,190.0


**6. Ordenar los autos por peso de manera descendente y mostrar los 5 más pesados.**

In [0]:
from pyspark.sql.functions import desc

df.orderBy(col('Weight').desc()).limit(5).display()


Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado
Pontiac Safari (sw),13.0,8,400.0,175.0,5140,12.0,71,US,Red,2019,Unico dueño
Chevrolet Impala,11.0,8,400.0,150.0,4997,14.0,73,US,Red,2019,Unico dueño
Dodge Monaco (sw),12.0,8,383.0,180.0,4955,11.5,71,US,Red,2019,Unico dueño
Mercury Marquis Brougham,12.0,8,429.0,198.0,4952,11.5,73,US,Red,2019,Unico dueño
Buick Electra 225 Custom,12.0,8,455.0,225.0,4951,11.0,73,US,Red,2019,Unico dueño


**7. Reemplazar los valores nulos en la columna "Horsepower" con el promedio de la misma columna.**

In [0]:
#Filtar los valores nulos de la columna Horsepower
nulos_hp = df.filter(col('Horsepower')==0)

prom_hp = df.select('Horsepower').agg(avg('Horsepower').alias('promedio_Horsepwer')).first()[0]
asigna_hp = df.withColumn('Horsepower', when(df['Horsepower'] == 0, prom_hp).otherwise(df['Horsepower']))
display(asigna_hp)


Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado
Chevrolet Chevelle Malibu,18.0,8,307.0,130.0,3504,12.0,70,US,Red,2019,Unico dueño
Buick Skylark 320,15.0,8,350.0,165.0,3693,11.5,70,US,Red,2019,Unico dueño
Plymouth Satellite,18.0,8,318.0,150.0,3436,11.0,70,US,Red,2019,Unico dueño
AMC Rebel SST,16.0,8,304.0,150.0,3433,12.0,70,US,Red,2019,Unico dueño
Ford Torino,17.0,8,302.0,140.0,3449,10.5,70,US,Red,2019,Unico dueño
Ford Galaxie 500,15.0,8,429.0,198.0,4341,10.0,70,US,Red,2019,Unico dueño
Chevrolet Impala,14.0,8,454.0,220.0,4354,9.0,70,US,Red,2019,Unico dueño
Plymouth Fury iii,14.0,8,440.0,215.0,4312,8.5,70,US,Red,2019,Unico dueño
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño
AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70,US,Red,2019,Unico dueño


**8. Crear una nueva columna "Power_to_Weight" que represente la relación entre "Horsepower" y "Weight" (Horsepower/Weight).**

In [0]:
#cambiamos valores 0 por valores 1 en la columna
eliminar_nulos_hp = df.withColumn('Horsepower', when(df['Horsepower']==0, 1).otherwise(df['Horsepower']))
relacion = eliminar_nulos_hp.withColumn('Power_to_Weight', col('Horsepower') / col('Weight'))
display(relacion)

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Power_to_Weight
Chevrolet Chevelle Malibu,18.0,8,307.0,130.0,3504,12.0,70,US,Red,2019,Unico dueño,0.037100456621004564
Buick Skylark 320,15.0,8,350.0,165.0,3693,11.5,70,US,Red,2019,Unico dueño,0.04467912266450041
Plymouth Satellite,18.0,8,318.0,150.0,3436,11.0,70,US,Red,2019,Unico dueño,0.043655413271245634
AMC Rebel SST,16.0,8,304.0,150.0,3433,12.0,70,US,Red,2019,Unico dueño,0.043693562481794346
Ford Torino,17.0,8,302.0,140.0,3449,10.5,70,US,Red,2019,Unico dueño,0.04059147579008408
Ford Galaxie 500,15.0,8,429.0,198.0,4341,10.0,70,US,Red,2019,Unico dueño,0.04561161022805805
Chevrolet Impala,14.0,8,454.0,220.0,4354,9.0,70,US,Red,2019,Unico dueño,0.050528249885163065
Plymouth Fury iii,14.0,8,440.0,215.0,4312,8.5,70,US,Red,2019,Unico dueño,0.049860853432282005
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño,0.05084745762711865
AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70,US,Red,2019,Unico dueño,0.04935064935064935


**9. Agrupar los autos por "Model" (año de modelo) y contar cuántos autos hay en cada año.**

In [0]:
df.groupBy(col('Model')).count().display()

Model,count
78,36
81,30
76,34
72,28
77,28
82,31
80,29
73,40
70,35
75,30


**10. Filtra y muestra los autos que tienen un consumo de combustible (MPG) mayor al promedio de todos los autos en el DataFrame.**

In [0]:
p_mpg = df.select(col('MPG')).agg(avg(col('MPG'))).first()[0]

mostrar = df.filter(col('MPG') > p_mpg)

display(mostrar)

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado
Toyota Corolla Mark ii,24.0,4,113.0,95.0,2372,15.0,70,Japan,Red,2019,Unico dueño
Datsun PL510,27.0,4,97.0,88.0,2130,14.5,70,Japan,Red,2019,Unico dueño
Volkswagen 1131 Deluxe Sedan,26.0,4,97.0,46.0,1835,20.5,70,Europe,Red,2019,Unico dueño
Peugeot 504,25.0,4,110.0,87.0,2672,17.5,70,Europe,Red,2019,Unico dueño
Audi 100 LS,24.0,4,107.0,90.0,2430,14.5,70,Europe,Red,2019,Unico dueño
Saab 99e,25.0,4,104.0,95.0,2375,17.5,70,Europe,Red,2019,Unico dueño
BMW 2002,26.0,4,121.0,113.0,2234,12.5,70,Europe,Red,2019,Unico dueño
Datsun PL510,27.0,4,97.0,88.0,2130,14.5,71,Japan,Red,2019,Unico dueño
Chevrolet Vega 2300,28.0,4,140.0,90.0,2264,15.5,71,US,Red,2019,Unico dueño
Toyota Corolla,25.0,4,113.0,95.0,2228,14.0,71,Japan,Red,2019,Unico dueño


**11. Agrupa los autos por la columna "Origin" y cuenta cuántos autos hay por cada origen.**

In [0]:
df.groupBy(col('Origin')).count().display()

Origin,count
Europe,73
US,254
Japan,79


**12. Para cada "Origin", encuentra el auto con mayor MPG y muestra su nombre, origen y consumo de combustible.**

In [0]:
max_mpg = df.groupBy(col('Origin')).agg(max(col('MPG')).alias('Max_MPG'))

df_alias1 = df.alias('df')

df_alias = df.alias("df")

mayor_car = df_alias1.join(max_mpg, (df_alias1["Origin"] == max_mpg["Origin"]) & (df_alias1["MPG"] == max_mpg["Max_MPG"])).select(df_alias1["Car"], df_alias1["Origin"], df_alias1["MPG"])
display(mayor_car)



Car,Origin,MPG
Mazda GLC,Japan,46.6
Volkswagen Rabbit C (Diesel),Europe,44.3
Plymouth Champ,US,39.0


**13. Crea una nueva columna "Weight_Normalized", donde normalices el peso de los autos usando la fórmula (Weight - min(Weight)) / max(Weight) - min(Weight)**

In [0]:
min_weight = df.select(min('Weight')).first()[0]
max_weight = df.select(max('Weight')).first()[0]

operacion = df.withColumn('Weight_Normalized', (col('Weight') - min_weight) / (max_weight - min_weight))

display(operacion)

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Weight_Normalized
Chevrolet Chevelle Malibu,18.0,8,307.0,130.0,3504,12.0,70,US,Red,2019,Unico dueño,0.536150
Buick Skylark 320,15.0,8,350.0,165.0,3693,11.5,70,US,Red,2019,Unico dueño,0.589736
Plymouth Satellite,18.0,8,318.0,150.0,3436,11.0,70,US,Red,2019,Unico dueño,0.516870
AMC Rebel SST,16.0,8,304.0,150.0,3433,12.0,70,US,Red,2019,Unico dueño,0.516019
Ford Torino,17.0,8,302.0,140.0,3449,10.5,70,US,Red,2019,Unico dueño,0.520556
Ford Galaxie 500,15.0,8,429.0,198.0,4341,10.0,70,US,Red,2019,Unico dueño,0.773462
Chevrolet Impala,14.0,8,454.0,220.0,4354,9.0,70,US,Red,2019,Unico dueño,0.777148
Plymouth Fury iii,14.0,8,440.0,215.0,4312,8.5,70,US,Red,2019,Unico dueño,0.765240
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño,0.797278
AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70,US,Red,2019,Unico dueño,0.634250


**14. Crea una nueva columna llamada "Power_Category" que clasifique los autos en:**

- "Bajo" si el Horsepower es menor a 100
- "Medio" si está entre 100 y 200
- "Alto" si es mayor a 200

In [0]:
category = df.withColumn('Power_Category', when(col('Horsepower') < 100, 'Bajo').when((col('Horsepower') >= 100) & (col('Horsepower') <=200), 'Medio').otherwise("Alto"))

display(category)

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Power_Category
Chevrolet Chevelle Malibu,18.0,8,307.0,130.0,3504,12.0,70,US,Red,2019,Unico dueño,Medio
Buick Skylark 320,15.0,8,350.0,165.0,3693,11.5,70,US,Red,2019,Unico dueño,Medio
Plymouth Satellite,18.0,8,318.0,150.0,3436,11.0,70,US,Red,2019,Unico dueño,Medio
AMC Rebel SST,16.0,8,304.0,150.0,3433,12.0,70,US,Red,2019,Unico dueño,Medio
Ford Torino,17.0,8,302.0,140.0,3449,10.5,70,US,Red,2019,Unico dueño,Medio
Ford Galaxie 500,15.0,8,429.0,198.0,4341,10.0,70,US,Red,2019,Unico dueño,Medio
Chevrolet Impala,14.0,8,454.0,220.0,4354,9.0,70,US,Red,2019,Unico dueño,Alto
Plymouth Fury iii,14.0,8,440.0,215.0,4312,8.5,70,US,Red,2019,Unico dueño,Alto
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño,Alto
AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70,US,Red,2019,Unico dueño,Medio


**15. Calcular el promedio de MPG por cada "Origin"**

In [0]:
df.groupBy('Origin').agg(avg(col('MPG'))).display()

Origin,avg(MPG)
Europe,26.745205479452057
US,19.688188976377948
Japan,30.450632911392397


**16. Encontrar el auto con el menor peso para cada "Cylinders"**

In [0]:
df.groupBy('Cylinders').agg(min(col('Weight')).alias('Peso_minimo')).display()


Cylinders,Peso_minimo
6,2472
3,2124
5,2830
4,1613
8,3086


**17. Crear una nueva columna "Efficiency_Category". Clasifica los autos según su MPG en tres categorías:**
- "Baja" si MPG < 20
- "Media" si 20 ≤ MPG < 30
- "Alta" si MPG ≥ 30

In [0]:
df.withColumn('Efficiency_Category', when(col('MPG') < 20 , 'Baja').when( (col('MPG') >= 20) & (col('MPG') < 30) , 'Media' ).otherwise('Alta')).display()

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Efficiency_Category
Chevrolet Chevelle Malibu,18.0,8,307.0,130.0,3504,12.0,70,US,Red,2019,Unico dueño,Baja
Buick Skylark 320,15.0,8,350.0,165.0,3693,11.5,70,US,Red,2019,Unico dueño,Baja
Plymouth Satellite,18.0,8,318.0,150.0,3436,11.0,70,US,Red,2019,Unico dueño,Baja
AMC Rebel SST,16.0,8,304.0,150.0,3433,12.0,70,US,Red,2019,Unico dueño,Baja
Ford Torino,17.0,8,302.0,140.0,3449,10.5,70,US,Red,2019,Unico dueño,Baja
Ford Galaxie 500,15.0,8,429.0,198.0,4341,10.0,70,US,Red,2019,Unico dueño,Baja
Chevrolet Impala,14.0,8,454.0,220.0,4354,9.0,70,US,Red,2019,Unico dueño,Baja
Plymouth Fury iii,14.0,8,440.0,215.0,4312,8.5,70,US,Red,2019,Unico dueño,Baja
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño,Baja
AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70,US,Red,2019,Unico dueño,Baja


**18. Contar cuántos autos hay por cada combinación de "Cylinders" y "Origin". Ordena el resultado en orden descendente.**

In [0]:
agrupar= df.groupBy(col('Cylinders'), col('Origin')).count()

resultado_desc = agrupar.orderBy(col("count").desc())

display(resultado_desc)


Cylinders,Origin,count
8,US,108
6,US,74
4,US,72
4,Japan,69
4,Europe,66
6,Japan,6
3,Japan,4
6,Europe,4
5,Europe,3


**19. Calcular la relación entre el peso y el caballaje de los autos.**
- Crea una nueva columna llamada "Weight_per_Horsepower".
- Esta columna debe contener el resultado de Weight / Horsepower.
- Muestra los autos con la mayor y menor relación de peso/caballaje.

In [0]:
crear_columna = df.withColumn('Weight_per_Horsepower', lit(col('Weight') / col('Horsepower')))

relacion_max = crear_columna.select(max(col('Weight_per_Horsepower'))).first()[0]
relacion_min = crear_columna.select(min(col('Weight_per_Horsepower'))).first()[0]

resultado_max = crear_columna.filter(col('Weight_per_Horsepower') == relacion_max)
resultado_min = crear_columna.filter(col('Weight_per_Horsepower') == relacion_min)

display(resultado_min)
display(resultado_max)



Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Weight_per_Horsepower
Buick Estate Wagon (sw),14.0,8,455.0,225.0,3086,10.0,70,US,Red,2019,Unico dueño,13.715555555555556


Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Weight_per_Horsepower
Volkswagen Dasher (diesel),43.4,4,90.0,48.0,2335,23.7,80,Europe,Red,2019,Unico dueño,48.645833333333336


**20. Encontrar los autos más eficientes dentro de cada grupo de cilindros.**
- Objetivo: Para cada número de cilindros (Cylinders), encontrar el auto con el mayor MPG (consumo de combustible).

In [0]:
agrupar_cilindros = df.groupBy(col('Cylinders')).agg(max(col('MPG')).alias('MPG_Maximo'))

df_alias = df.alias("df")

#estructura basica del join
# df1.join(df2, (df1['columna1] == df2['columna1]), 'tipo_de_join')

# en este caso estamos comparando cual dato esta igual en las columnas CYLINDERS y MPG de los dos dataframe
# para poder mostrar depues, la columna CAR
# Ya que si no lo hacemos de esa manera, solo mostrara las columnas CYLINDERS y MPG. 
efficient_cars = df_alias.join(agrupar_cilindros, (df_alias["Cylinders"] == agrupar_cilindros["Cylinders"]) & (df_alias["MPG"] == agrupar_cilindros["MPG_Maximo"])).select(df_alias["Car"], df_alias["Cylinders"], df_alias["MPG"])
display(efficient_cars)

Car,Cylinders,MPG
Mazda GLC,4,46.6
Audi 5000s (diesel),5,36.4
Mazda RX-7 GS,3,23.7
Oldsmobile Cutlass LS,8,26.6
Oldsmobile Cutlass Ciera (diesel),6,38.0


**21. Calcular el promedio de caballaje por origen y categoría de peso**
- Objetivo: Agrupar los autos en tres categorías de peso (Light, Medium, Heavy) y calcular el promedio de caballaje (Horsepower) por Origin y categoría de peso.

Light < 2000 ;
2000 <= Medium < 4000 ;
Heavy >= 4000

In [0]:
new_column = df.withColumn('Category_Weight', when(col('Weight') < 2000, 'Light').when((col('Weight') >= 2000) & (col('Weight') < 4000), 'Medium' ).otherwise('Heavy'))

group_origin_category = new_column.groupBy(col('Category_Weight'), col('Origin')).agg(avg(col('Horsepower')).alias('Prom_caballaje'))

display(group_origin_category)

Category_Weight,Origin,Prom_caballaje
Heavy,US,166.92537313432837
Light,Japan,63.26086956521739
Medium,Japan,86.64285714285714
Light,Europe,58.1764705882353
Medium,Europe,85.03571428571429
Light,US,70.0
Medium,US,101.15300546448087


**22. Determinar la correlación entre MPG y Horsepower**
- Objetivo: Calcular la correlación entre MPG y Horsepower para analizar si los autos más potentes consumen más combustible.

In [0]:
correlacion = df.stat.corr('MPG', 'Horsepower')

display(correlacion)

-0.7266702031896445

**23. Encontrar el auto con la mayor eficiencia de combustible considerando peso y caballaje**
- Objetivo: Crear una nueva métrica llamada Efficiency_Score = MPG / (Weight / Horsepower), y encontrar el auto con el mayor valor de Efficiency_Score.

In [0]:


efficiency = df.withColumn('Efficiency_Score', expr('MPG / (Weight / Horsepower)'))
mayor_efficinency = efficiency.orderBy(col('Efficiency_Score').desc()).limit(1)

display(mayor_efficinency)

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Efficiency_Score
Honda Civic 1500 gl,44.6,4,91.0,67.0,1850,13.8,80,Japan,Red,2019,Unico dueño,1.6152432432432433


**24. Analizar la evolución del MPG promedio por año**
- Objetivo: Si la base de datos tiene una columna Year, calcular cómo ha cambiado el consumo promedio de combustible (MPG) con el tiempo.

In [0]:
evolution = df.groupBy('Model').agg(avg('MPG').alias('Prom_MPG'))

order_model = evolution.orderBy(col('Model'))

display(order_model)

Model,Prom_MPG
70,14.657142857142857
71,20.517241379310345
72,18.714285714285715
73,17.1
74,22.703703703703702
75,20.266666666666666
76,21.573529411764707
77,23.375
78,24.061111111111114
79,25.09310344827585


**25. Predecir tendencias del consumo de combustible**
- Objetivo: Analizar si hay una tendencia creciente o decreciente en el consumo promedio de combustible (MPG) en función de los años.

In [0]:
tendencia = df.groupBy(col('Model')).agg(avg(col('MPG')))

resultado_ordenado = tendencia.orderBy(col('Model'))

display(resultado_ordenado)

Model,avg(MPG)
70,14.657142857142857
71,20.517241379310345
72,18.714285714285715
73,17.1
74,22.703703703703702
75,20.266666666666666
76,21.573529411764707
77,23.375
78,24.061111111111114
79,25.09310344827585


**26. Encontrar los 3 autos más rápidos por cada origen**
- Objetivo: Identificar los tres autos con mayor caballaje (Horsepower) en cada grupo de Origin.

In [0]:
grupo_origen = Window.partitionBy('Origin').orderBy(col("Horsepower").desc())

ranking = df.withColumn('Ranking', row_number().over(grupo_origen))
three_ranking = ranking.filter(col('Ranking') <= 3)
display(three_ranking)

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Ranking
Peugeot 604sl,16.2,6,163.0,133.0,3410,15.8,78,Europe,Red,2019,Unico dueño,1
Volvo 264gl,17.0,6,163.0,125.0,3140,13.6,78,Europe,Red,2019,Unico dueño,2
Mercedes-Benz 280s,16.5,6,168.0,120.0,3820,16.7,76,Europe,Red,2019,Unico dueño,3
Datsun 280-ZX,32.7,6,168.0,132.0,2910,11.4,80,Japan,Red,2019,Unico dueño,1
Toyota Mark II,20.0,6,156.0,122.0,2807,13.5,73,Japan,Red,2019,Unico dueño,2
Datsun 810 Maxima,24.2,6,146.0,120.0,2930,13.8,81,Japan,Red,2019,Unico dueño,3
Pontiac Grand Prix,16.0,8,400.0,230.0,4278,9.5,73,US,Red,2019,Unico dueño,1
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño,2
Buick Estate Wagon (sw),14.0,8,455.0,225.0,3086,10.0,70,US,Red,2019,Unico dueño,3


**27.  Normalización del Peso de los Autos**
- Objetivo: Crear una nueva columna llamada Weight_Normalized, donde el peso de los autos (Weight) se normalice utilizando la siguiente fórmula:
Weight - min(Weight) / max(Weight) - min(Weight)

In [0]:
max_peso = df.select(max(col('Weight'))).first()[0]
min_peso = df.select(min(col('Weight'))).first()[0]

normalized_column = df.withColumn('Weight_normalized', lit((col('Weight') - min_peso) / (max_peso - min_peso)))

display(normalized_column)

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Weight_normalized
Chevrolet Chevelle Malibu,18.0,8,307.0,130.0,3504,12.0,70,US,Red,2019,Unico dueño,0.536150
Buick Skylark 320,15.0,8,350.0,165.0,3693,11.5,70,US,Red,2019,Unico dueño,0.589736
Plymouth Satellite,18.0,8,318.0,150.0,3436,11.0,70,US,Red,2019,Unico dueño,0.516870
AMC Rebel SST,16.0,8,304.0,150.0,3433,12.0,70,US,Red,2019,Unico dueño,0.516019
Ford Torino,17.0,8,302.0,140.0,3449,10.5,70,US,Red,2019,Unico dueño,0.520556
Ford Galaxie 500,15.0,8,429.0,198.0,4341,10.0,70,US,Red,2019,Unico dueño,0.773462
Chevrolet Impala,14.0,8,454.0,220.0,4354,9.0,70,US,Red,2019,Unico dueño,0.777148
Plymouth Fury iii,14.0,8,440.0,215.0,4312,8.5,70,US,Red,2019,Unico dueño,0.765240
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño,0.797278
AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70,US,Red,2019,Unico dueño,0.634250


**28. Categorizar autos según su eficiencia**
- Objetivo: Crear una nueva columna "Efficiency_Category" que clasifique los autos según su consumo de combustible (MPG):

- "Bajo" si MPG < 15
- "Medio" si 15 ≤ MPG ≤ 25
- "Alto" si MPG > 25

In [0]:
category_Efficiency = df.withColumn('Efficiency_Category', when(col('MPG') < 15, 'Bajo').when((col('MPG') >=15 ) & (col('MPG') <= 25), 'Medio').otherwise('Alto'))

result_category = category_Efficiency.orderBy('MPG')

display(result_category)

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado,Efficiency_Category
Citroen DS-21 Pallas,0.0,4,133.0,115.0,3090,17.5,70,Europe,Red,2019,Unico dueño,Bajo
Chevrolet Chevelle Concours (sw),0.0,8,350.0,165.0,4142,11.5,70,US,Red,2019,Unico dueño,Bajo
Ford Torino (sw),0.0,8,351.0,153.0,4034,11.0,70,US,Red,2019,Unico dueño,Bajo
Plymouth Satellite (sw),0.0,8,383.0,175.0,4166,10.5,70,US,Red,2019,Unico dueño,Bajo
AMC Rebel SST (sw),0.0,8,360.0,175.0,3850,11.0,70,US,Red,2019,Unico dueño,Bajo
Ford Mustang Boss 302,0.0,8,302.0,140.0,3353,8.0,70,US,Red,2019,Unico dueño,Bajo
Volkswagen Super Beetle 117,0.0,4,97.0,48.0,1978,20.0,71,Europe,Red,2019,Unico dueño,Bajo
Saab 900s,0.0,4,121.0,110.0,2800,15.4,81,Europe,Red,2019,Unico dueño,Bajo
Hi 1200D,9.0,8,304.0,193.0,4732,18.5,70,US,Red,2019,Unico dueño,Bajo
Ford F250,10.0,8,360.0,215.0,4615,14.0,70,US,Red,2019,Unico dueño,Bajo


**29. Comparar el peso promedio entre autos de 4, 6 y 8 cilindros**
- Objetivo: Calcular el peso promedio (Weight) de los autos según su número de cilindros (Cylinders).

In [0]:
group_cylinders = df.groupBy(col('Cylinders')).agg(avg(col('Weight')).alias('Prom_Weight'))

result_order = group_cylinders.orderBy(col('Cylinders'))

selection_cylinders = result_order.filter((col('Cylinders') == 4) | (col('Cylinders') == 6) | (col('Cylinders') == 8))
display(selection_cylinders)

Cylinders,Prom_Weight
4,2312.6860
6,3198.2262
8,4105.1944


# **Trabajando SQL con pyspark**

In [0]:
df.createOrReplaceTempView("cars")

spark.sql("SELECT * FROM cars").display()

Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin,Color,Modelo,Estado
Chevrolet Chevelle Malibu,18.0,8,307.0,130.0,3504,12.0,70,US,Red,2019,Unico dueño
Buick Skylark 320,15.0,8,350.0,165.0,3693,11.5,70,US,Red,2019,Unico dueño
Plymouth Satellite,18.0,8,318.0,150.0,3436,11.0,70,US,Red,2019,Unico dueño
AMC Rebel SST,16.0,8,304.0,150.0,3433,12.0,70,US,Red,2019,Unico dueño
Ford Torino,17.0,8,302.0,140.0,3449,10.5,70,US,Red,2019,Unico dueño
Ford Galaxie 500,15.0,8,429.0,198.0,4341,10.0,70,US,Red,2019,Unico dueño
Chevrolet Impala,14.0,8,454.0,220.0,4354,9.0,70,US,Red,2019,Unico dueño
Plymouth Fury iii,14.0,8,440.0,215.0,4312,8.5,70,US,Red,2019,Unico dueño
Pontiac Catalina,14.0,8,455.0,225.0,4425,10.0,70,US,Red,2019,Unico dueño
AMC Ambassador DPL,15.0,8,390.0,190.0,3850,8.5,70,US,Red,2019,Unico dueño
